# 11 - Distance Manipulation and Analysis:

### In this tutorial we will learn about bond distance manipulation, calculation and visualization routines within Architector.

**(A)** Generating with fixed metal-ligand bond lengths in an Iron Hexa-aqua complex.

**(B)** Visualizing tricks for bond lengths.

**(C)** Tabulating and plotting bond lengths.

## Starting from **(A)**: 
Importing necessary functions!

Let's build a reference set of Fe-Hexa-Aqua Structures

In [ ]:
from architector import (build_complex,
                        view_structures,
                        convert_io_molecule)
import architector.io_ptable as io_ptable # Contains ligand references

In [ ]:
inputDict = {'core':{'metal':'Fe','coreCN':6}, #Specify metal coordination number (CN)
            'ligands':['water']*6, # Specify what is filling the coordination environment
            'parameters':{}} # No additional parameters needed for default
fe_h2o_6_out = build_complex(inputDict) # Now just build using the dictionary!

In [ ]:
# Now, we can visualize interatomic distances
view_structures(fe_h2o_6_out,vis_distances=True)

Now, we can create elonged metal-ligand distances by specifying our own ligand definition

In [ ]:
stretch_water_lig = io_ptable.ligands_dict['water'].copy()
stretch_water_lig

In [ ]:
# Now, we can add metal-distance constraints 
# These take the form {coordinatng_atom_index:distance}
stretch_water_lig['ca_metal_dist_constraints'] = {0:2.6}

In [ ]:
# Now, we can build with 2 ligands stretched!
inputDict2 = {'core':{'metal':'Fe','coreCN':6}, #Specify metal coordination number (CN)
            'ligands':['water']*4+[stretch_water_lig]*2, # Specify what is filling the coordination environment
            'parameters':{'relax':True,'return_full_complex_class':True}} # No additional parameters needed for default
fe_h2o_6_out2 = build_complex(inputDict2) # Now just build using the dictionary!

In [ ]:
# Note that 2 waters are now fixed at ~2.58 Angstroms while the rest have different bond lengths from above!
view_structures(fe_h2o_6_out2, vis_distances=True)

## Beyond bond lengths shown in (A), we can see more diverse visualizations in **(B)**

First, we will get the last molecule from (A):

In [ ]:
example_mol = convert_io_molecule(fe_h2o_6_out2[list(fe_h2o_6_out2.keys())[0]]['mol2string'])
view_structures(example_mol,labelinds=True) # Start by labeling indices

In [ ]:
view_structures(example_mol,
                vis_distances = True, # For vis_distances=True, the metal(s) in a system are selected for calculation.
                distradius=4, # By Specifying distradius, it will show all bonds within a cutoff of the requested atom
                ) 

This looks very crowded! How can we see these individual distances better.

In [ ]:
view_structures(example_mol,
                vis_distances = True, # For vis_distances=True, the metal(s) in a system are selected for calculation.
                distradius=4, # By Specifying distradius, it will show all bonds within a cutoff of the requested atom
                distvisradius=0.1, # distvisradius makes the bond visualization smaller
                distlabelposit=0.9, # distlabelposit shifts the labels a fraction of the bond length (1 closer to end atom, 0 closer to start atom)
                distopacity=1, # distopacity shows how transparent/dark the bonds are drawn
                labelsize=9, # labelsize changes the text size
                representation='sphere', # Will only show spheres instead of bonds.
                ) 

Looks better, but can we only do metal-ligand distances? No!

In [ ]:
view_structures(example_mol,
                vis_distances = 1, # For vis_distances=int, the atom with index of int is used as the center for plotting distances.
                distradius=4, # By Specifying distradius, it will show all bonds within a cutoff of the requested atom
                distvisradius=0.1, # distvisradius makes the bond visualization smaller
                distlabelposit=0.9, # distlabelposit shifts the labels a fraction of the bond length (1 closer to end atom, 0 closer to start atom)
                distopacity=1, # distopacity shows how transparent/dark the bonds are drawn
                labelsize=9, # labelsize changes the text size
                representation='sphere', # Will only show spheres instead of bonds.
                ) 

In [ ]:
view_structures(example_mol,
                vis_distances = [1,4], # For vis_distances=list, the atoms with indices of the center(s) are used for plotting distances.
                distradius=4, # By Specifying distradius, it will show all bonds within a cutoff of the requested atom
                distvisradius=0.1, # distvisradius makes the bond visualization smaller
                distlabelposit=0.9, # distlabelposit shifts the labels a fraction of the bond length (1 closer to end atom, 0 closer to start atom)
                distopacity=1, # distopacity shows how transparent/dark the bonds are drawn
                labelsize=9, # labelsize changes the text size
                representation='sphere', # Will only show spheres instead of bonds.
                ) 

This looks better, and interesting, but distinguishing these can be tricky! Let's do just one more tweak.

In [ ]:
view_structures(example_mol,
                vis_distances = [1,4], # For vis_distances=list, the atoms with indices of the center(s) are used for plotting distances.
                distradius=4, # By Specifying distradius, it will show all bonds within a cutoff of the requested atom
                distvisradius=0.1, # distvisradius makes the bond visualization smaller
                distlabelposit=0.9, # distlabelposit shifts the labels a fraction of the bond length (1 closer to end atom, 0 closer to start atom)
                distopacity=1, # distopacity shows how transparent/dark the bonds are drawn
                distcolor=['red','blue'], # If a list of color labels is passed, they can be manually changed to represent the indices selected in the vis_distances list!
                labelsize=9, # labelsize changes the text size
                representation='sphere', # Will only show spheres instead of bonds.
                ) 

Lastly, we may want to add only specific atom-atom distances.

In [ ]:
view_structures(example_mol,
                vis_distances = True,
                distatompairs= [[0,1],[1,2]] # Here, only specific pairs of atoms are visualized for clarity.
                ) 

## Beyond visualizations in (B), we may want to simply plot or tabulate the bond distances.
## This leads us to **(C)**:

Here, we can immediately obtain the bond distances in a table with one call from Architector!

In [ ]:
df = example_mol.get_dists()
df

Note that the atom_pair corresponds to the distances, and all ofhter information including the sum of covalent radii in angstroms are included in the table.

Further, note that for ligands, both the smiles string AND the index of the atom in the smiles string are computed.

We might be more interested in longer distances as before and not use the metal as the index.

In [ ]:
df = example_mol.get_dists(radius=4,ref_ind=2)
df

Note that the bond_type and atom_symbols for the Fe-H bonds have been flagged as "implicit_bonds". Because they were not included in the graph.

Now, pandas/matplotlib can be used to generate nice plots illustrating the distance distributions!

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
g = df.groupby('atom_symbols')['distance']
axes = g.plot(kind='hist',bins=np.arange(0.5,4.5,0.5),alpha=0.5,align='left')
for i, (groupname, group) in enumerate(g):
    axes[i].set_label(groupname)
plt.legend()
plt.xlabel('Interatomic Distance ($\AA$)')
plt.title('Distance Plot')

Note, if you want to save/export the data simply execute:

`df.to_csv('example.csv',index=False)`

## Summary

### In this tutorial we learned about bond distance manipulation, calculation and visualization routines within Architector in 3 example sections:

**(A)** Generated  Iron Hexa-aqua complexes with fixed metal-ligand bond lengths.

**(B)** Visualized bond distances with in-build routine.

**(C)** Tabulated and plotted bond lengths.